In [1]:
from torchvision import transforms as T

class SimpsonsTransforms(T.Compose):
    def __init__(self, phase):
        self.phase = phase
        self.transforms = {
            'train': [
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ],
            'val': [
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ],
            'test': [
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ]
        }
        
        super().__init__(self.transforms[self.phase])

In [2]:
from torchvision.datasets import ImageFolder

class SimpsonsImageFolder(ImageFolder):
    def __init__(self, root, phase):
        self.root = f"{root}/{phase}"
        self.phase = phase
        self.transform = SimpsonsTransforms(phase=phase)
        
        super().__init__(self.root, self.transform)

In [3]:
from pytorch_lightning import LightningDataModule

from torch.utils.data import DataLoader

class SimpsonsDataModule(LightningDataModule):
    def __init__(self, dataset_path, batch_size):
        super().__init__()
        self.dataset_path = dataset_path
        self.batch_size = batch_size
    
    def train_dataloader(self):
        self.train_imagefolder = SimpsonsImageFolder(root=self.dataset_path, 
                                                     phase='train')
        return DataLoader(dataset=self.train_imagefolder,
                          batch_size=self.batch_size,
                          num_workers=2)
    
    def val_dataloader(self):
        self.val_imagefolder = SimpsonsImageFolder(root=self.dataset_path,
                                                   phase='val')
        return DataLoader(dataset=self.val_imagefolder,
                          batch_size=self.batch_size,
                          num_workers=2)
    
    def test_dataloader(self):
        self.test_imagefolder = SimpsonsImageFolder(root=self.dataset_path,
                                                    phase='test')
        return DataLoader(dataset=self.test_imagefolder,
                          batch_size=self.batch_size,
                          num_workers=2)

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule
from pytorch_lightning.metrics.functional import accuracy


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)


class SimpsonsNet(LightningModule):
    def __init__(self):
        super(SimpsonsNet, self).__init__()
        
        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            Flatten(),
            nn.Dropout(p=0.1),
            nn.Linear(32 * 56 * 56, 64),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.1),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        out = self.sequential(x)
        return F.log_softmax(out, dim=-1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        return loss

    def _evaluate(self, batch, batch_idx, stage=None):
        x, y = batch
        out = self.forward(x)
        logits = F.log_softmax(out, dim=-1)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=-1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_acc', acc, prog_bar=True)

        return loss, acc

    def validation_step(self, batch, batch_idx):
        return self._evaluate(batch, batch_idx, 'val')[0]

    def test_step(self, batch, batch_idx):
        loss, acc = self._evaluate(batch, batch_idx, 'test')
        self.log_dict({'test_loss': loss, 'test_acc': acc})

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=2e-3)

In [5]:
data_module = SimpsonsDataModule(dataset_path="../dataset", batch_size=32)

In [6]:
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

In [7]:
model = SimpsonsNet()

In [8]:
# import wandb
# wandb.login()

In [9]:
# from pytorch_lightning.loggers import WandbLogger

# wandb_logger = WandbLogger()

In [10]:
import pytorch_lightning as pl

trainer = pl.Trainer(gpus=1, progress_bar_refresh_rate=20, max_epochs=2)    
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name       | Type       | Params
------------------------------------------
0 | sequential | Sequential | 6.4 M 
------------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
25.713    Total estimated model params size (MB)
/home/alvarobartt/miniconda3/envs/deeplearning/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/alvarobartt/miniconda3/envs/deeplearning/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1